# Building Graphs in wc_rules

In `wc_rules`, we are essentially constructing graph representations of chemical entities. The basic building blocks of the graph are 
* nodes, which include instances of molecules, sites and site-relations, and,
* edges, which are bidirectional relations between molecules, sites and site-relations. 

To build this graph, one needs to know about :
* subclasses
* instances
* relations
* instance attribute methods

### Subclasses

Objects such as molecules, sites and site-relations are derived from generic base classes provided in `wc_rules`. To create various _types_ of these objects, one can simply subclass them as often as needed.

Below, we create the type hierarchy
* `Molecule -> A -> A1`

where `A` is a subclass of molecule and `A1` is a subclass of `A`

In [1]:
from wc_rules.chem2 import Molecule,Site

class A(Molecule):
    pass

class A1(A):
    pass

### Instances

Instances are created by calling the constructor of the class. These constitute the nodes of the graph.

In [2]:
a1_001 = A1()
a1_001

Instances typically have scalar attributes that can be set during construction. For example, all instances have an `id` attribute.

In [3]:
a1_001 = A1(id='instance_001_of_A1')
a1_001.id

'instance_001_of_A1'

Instances can be type-checked against any parent class.

In [4]:
isinstance(a1_001,A1) and isinstance(a1_001,A) and isinstance(a1_001,Molecule)

True

### Relations

Instances have _relations_ to each other, which constitute the edges of the graph. Relations are managed using pairs of attributes on each of the respective classes. For example, `Molecule` has a `sites` attribute that holds a list of site instances. `Site` has a `molecule` attribute that holds a single instance of a molecule. 

In [5]:
m,s = Molecule(), Site()
m.sites, s.molecule

([], None)

In [6]:
m.sites.append(s)
m.sites

In [7]:
s.molecule

The convention for attribute name is typically to use a singular name such as `molecule` if it refers to a single instance or a plural name such as `sites` if it refers to a list of instances.

### Instance Attribute Methods

`wc_rules` provides setters, getters and unsetters for various instance attributes. For example, for the `id` attribute, we have `get_id()` and `set_id(id)`.

In [8]:
m = Molecule()
m.set_id('m_001')
m.get_id()

'm_001'

Typically, the following naming conventions are followed for getters, setters and unsetters:

|Attribute type|Getter |Setter |Unsetter|
|---|---|---|---|---|
|Scalar|`get_*`|`set_*`| |
|Object reference|`get_*`|`set_*`|`unset_*`|
|List of object references|`get_*`|`add_*`|`remove_*`|

Setters always "return self", so they can be chained indefinitely. For example,

In [9]:
m = Molecule().set_id('m_001').add_sites( Site(), Site() )
m.sites

Additional points to remember about instance attribute methods:
* Each class may have additional inbuilt getters and setters with custom functionality.
* The methods are inherited during subclassing.
* Setters managing class relations can have redundant functionality, e.g., `Molecule.add_sites()` and `Site.set_molecule()` will have the same effect of establishing relations between molecules and sites.